# Data Preprocessing with Pandas

## Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Import data

Data is generated from [Australia Bureau of statistics](http://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/6416.0Mar%202015?OpenDocument), some cells are removed (set to NaN) manually in order to serve this notebook.

In [2]:
df = pd.read_csv('../asset/Median Price of Established House Transfers.txt', sep='\t') # row 3 has a null value
df.head()

,City,Quarter,Price
0,Sydney,Q1,700.0
1,Rest of NSW,Q1,370.0
2,Melbourne,Q1,525.0
3,Rest of Vic.,Q1,NaN
4,Brisbane,Q1,460.0


## Finding and Handling Missing Data

In [3]:
# find rows that Price is null
df[pd.isnull(df['Price'])]

,City,Quarter,Price
3,Rest of Vic.,Q1,NaN
13,Rest of NT,Q1,NaN
21,Adelaide,Q2,NaN
27,Darwin,Q2,NaN
40,Hobart,Q3,NaN
49,Brisbane,Q4,NaN


In [4]:
index_with_null = df[pd.isnull(df['Price'])].index
index_with_null

Int64Index([3, 13, 21, 27, 40, 49], dtype='int64')

We can specify a value (e.g., 0) to replace those null values, through the fillna() method

In [5]:
df2 = df.fillna(0) # price value of row 3 is set to 0.0
df2.loc[index_with_null]
# df2.index in index_with_null

,City,Quarter,Price
3,Rest of Vic.,Q1,0.0
13,Rest of NT,Q1,0.0
21,Adelaide,Q2,0.0
27,Darwin,Q2,0.0
40,Hobart,Q3,0.0
49,Brisbane,Q4,0.0


We can also propagate non-null values forward or backward

In [6]:
df2 = df.fillna(method='pad', axis=0)
df2.head() # The price of row 3 is the same as that of row 2

,City,Quarter,Price
0,Sydney,Q1,700.0
1,Rest of NSW,Q1,370.0
2,Melbourne,Q1,525.0
3,Rest of Vic.,Q1,525.0
4,Brisbane,Q1,460.0


We can even drop the rows (or columns) with null values

In [7]:
df2 = df.dropna(axis=0) # if axis = 1 then the column will be dropped
df2.head() # Note that row 3 is dropped

,City,Quarter,Price
0,Sydney,Q1,700.0
1,Rest of NSW,Q1,370.0
2,Melbourne,Q1,525.0
4,Brisbane,Q1,460.0
5,Rest of Qld.,Q1,400.0


Obviously, none of the above solutions are appropriate.

A better way to deal with the null value is to replace them with the mean value of the prices of the corresponding city over the whole year.

In [17]:
df["Price"] = df.groupby("City").transform(lambda x: x.fillna(x.mean()))
df.loc[index_with_null]

,City,Quarter,Price,Bin,Price-Smoothing-mean,Price-Smoothing-max
3,Rest of Vic.,Q1,294.666667,Very Low,270.805556,302.0
13,Rest of NT,Q1,422.500000,Medium,415.644444,427.5
21,Adelaide,Q2,420.433333,Medium,415.644444,427.5
27,Darwin,Q2,579.166667,Very High,634.763889,835.0
40,Hobart,Q3,359.000000,Low,372.416667,387.5
49,Brisbane,Q4,468.633333,High,519.961111,555.0


## Binning

### Equal-width Partitioning

We use the table with all null values filled

In [9]:
pd.cut(df['Price'],5).head() # equally partition Price into 5 bins

0      (597.0, 716.0]
1      (359.0, 478.0]
2      (478.0, 597.0]
3    (239.405, 359.0]
4      (359.0, 478.0]
Name: Price, dtype: category
Categories (5, interval[float64]): [(239.405, 359.0] < (359.0, 478.0] < (478.0, 597.0] < (597.0, 716.0] < (716.0, 835.0]]

In [10]:
# We could label the bins and add new column
df['Bin'] = pd.cut(df['Price'],5,labels=["Very Low","Low","Medium","High","Very High"])
df.head()

,City,Quarter,Price,Bin
0,Sydney,Q1,700.000000,High
1,Rest of NSW,Q1,370.000000,Low
2,Melbourne,Q1,525.000000,Medium
3,Rest of Vic.,Q1,294.666667,Very Low
4,Brisbane,Q1,460.000000,Low


### Equal-depth Partitioining

In [11]:
pd.qcut(df['Price'],5).head() # Note the difference from the Equal-width Partitioning case

0      (555.1, 835.0]
1      (336.4, 395.0]
2      (440.5, 555.1]
3    (239.999, 336.4]
4      (440.5, 555.1]
Name: Price, dtype: category
Categories (5, interval[float64]): [(239.999, 336.4] < (336.4, 395.0] < (395.0, 440.5] < (440.5, 555.1] < (555.1, 835.0]]

In [12]:
# Let's check the depth of each bin
df['Bin'] = pd.qcut(df['Price'],5,labels=["Very Low","Low","Medium","High","Very High"])
df.groupby('Bin').size()

Bin
Very Low     12
Low          12
Medium       12
High         12
Very High    12
dtype: int64

## Smoothing

### Smoothing by Bin Means

In [13]:
df.head()

,City,Quarter,Price,Bin
0,Sydney,Q1,700.000000,Very High
1,Rest of NSW,Q1,370.000000,Low
2,Melbourne,Q1,525.000000,High
3,Rest of Vic.,Q1,294.666667,Very Low
4,Brisbane,Q1,460.000000,High


In [14]:
df['Price-Smoothing-mean'] = df.groupby('Bin')['Price'].transform('mean')
df.head()

,City,Quarter,Price,Bin,Price-Smoothing-mean
0,Sydney,Q1,700.000000,Very High,634.763889
1,Rest of NSW,Q1,370.000000,Low,372.416667
2,Melbourne,Q1,525.000000,High,519.961111
3,Rest of Vic.,Q1,294.666667,Very Low,270.805556
4,Brisbane,Q1,460.000000,High,519.961111


### Smoothing by Bin Max

In [15]:
df['Price-Smoothing-max'] = df.groupby('Bin')['Price'].transform('max')
df.head()

,City,Quarter,Price,Bin,Price-Smoothing-mean,Price-Smoothing-max
0,Sydney,Q1,700.000000,Very High,634.763889,835.0
1,Rest of NSW,Q1,370.000000,Low,372.416667,387.5
2,Melbourne,Q1,525.000000,High,519.961111,555.0
3,Rest of Vic.,Q1,294.666667,Very Low,270.805556,302.0
4,Brisbane,Q1,460.000000,High,519.961111,555.0


## Normalization

In [18]:
df = pd.read_csv('../asset/Median Price of Established House.txt', sep='\t')
df.head()

,City,Q1,Q2,Q3,Q4
0,Sydney,700.0,750.0,750.0,835.0
1,Rest of NSW,370.0,375.0,373.0,385.0
2,Melbourne,525.0,555.5,540.0,580.0
3,Rest of Vic.,300.0,295.0,287.0,302.0
4,Brisbane,460.0,474.9,471.0,485.0


### Standard Scaler

In [20]:
from sklearn import preprocessing

min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(df[df.columns[1:5]]) # we need to remove the first column


df_standard = pd.DataFrame(x_scaled)
df_standard.insert(0, 'City', df.City)
df_standard

ModuleNotFoundError: No module named 'scipy'

## Robust Scaler

In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.RobustScaler()
x_scaled = min_max_scaler.fit_transform(df[df.columns[1:5]]) # we need to remove the first column

df_robust = pd.DataFrame(x_scaled)
df_robust.insert(0, 'City', df.City)
df_robust

## Histograms

In [ ]:
df = pd.read_csv('./asset/Median Price of Established House.txt', sep='\t')
df.head()

In [ ]:
# use bins=x to control the number of bins
df.hist(column=['Q1','Q3'],bins=6,alpha=0.5,figsize=(16, 6))

## Scatter

In [ ]:
df.plot.scatter(x='Q1', y='Q3');

Scatter matrix provide a better way to discover the relationships in data

In [ ]:
from pandas.tools.plotting import scatter_matrix

scatter_matrix(df, alpha=0.9, figsize=(12, 12), diagonal='hist') # set the diagonal figures to be histograms

## Further reading

* https://blog.modeanalytics.com/python-data-cleaning-libraries/?utm_campaign=Data%2BElixir&utm_medium=email&utm_source=Data_Elixir_94